# Deep Network
This example is adding more layers to the NN. Starting from the intermediate NN construct. See notebook.

## Set the seed

In [1]:
import numpy as np
np.random.seed(42)

#### Load dependencies
Keras is the python API to access tensorflow

In [2]:
import keras
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers.normalization import BatchNormalization
from keras.optimizers import SGD
from keras import regularizers

Using TensorFlow backend.
/opt/conda/lib/python3.6/importlib/_bootstrap.py:205: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


#### Prepare data

In [3]:
(X_train,y_train),(X_test,y_test)= mnist.load_data()
X_train = X_train.reshape(60000,784).astype('float32')
X_test = X_test.reshape(10000,784).astype('float32')
# Apply matrix scalar division to bring value from 0 to 1
X_train /=255
X_test /= 255
# There are 10 digits to classify
n_classes=10
y_train = keras.utils.to_categorical(y_train,n_classes)
y_test = keras.utils.to_categorical(y_test,n_classes)


## Design deep neural network
Define 3 layers with 64 inputs. Deep begins at 3 layers
Tuning initialization is one of the parameters to play with. Here the Initialization in keras is using glorot_uniform, which is very good. Other values: glorot_normal. We need to add L2 regularization to the weight to control / prevent overfitting. Normally it is not necessary to regularize the bias or at the activity level, weight should be enough. The input parameter for L2 reg is a scale factor.
Dropout consists in randomly setting a fraction rate of input units to 0 at each update during training time, which helps prevent overfitting. 
BatchNormalization is making normalization a part of the model architecture and performing the normalization for each training mini-batch. Batch Normalization allows us to use much higher learning rates and be less careful about initialization. It also acts as a regularizer, in some cases eliminating the need for Dropout.

In [11]:
model = Sequential()
model.add(Dense(64,activation='relu',input_shape=(784,),
                kernel_initializer='glorot_uniform', kernel_regularizer=regularizers.l2(0.01)))
model.add(BatchNormalization())
model.add(Dropout(0.5))
model.add(Dense(64,activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.5))
model.add(Dense(64,activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.5))
model.add(Dense(10,activation='softmax'))

In [12]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_7 (Dense)              (None, 64)                50240     
_________________________________________________________________
batch_normalization_1 (Batch (None, 64)                256       
_________________________________________________________________
dropout_3 (Dropout)          (None, 64)                0         
_________________________________________________________________
dense_8 (Dense)              (None, 64)                4160      
_________________________________________________________________
batch_normalization_2 (Batch (None, 64)                256       
_________________________________________________________________
dropout_4 (Dropout)          (None, 64)                0         
_________________________________________________________________
dense_9 (Dense)              (None, 10)                650       
Total para

#### Compile the model and train
Specifying the cost function to use, the metric to optimize and the type of optimization. Could be gardiant descent by setting optimizer=SGD(lr=0.1). Adam brings the capability to decrease of the learning rate ver iteration automatically.

In [13]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(X_train, y_train, batch_size=128, epochs=50, verbose=1, validation_data=(X_test, y_test))

Train on 60000 samples, validate on 10000 samples
Epoch 1/50
60000/60000 [==============================] - 3s 46us/step - loss: 1.3175 - acc: 0.7374 - val_loss: 0.4970 - val_acc: 0.9218
Epoch 2/50
60000/60000 [==============================] - 2s 41us/step - loss: 0.6169 - acc: 0.8686 - val_loss: 0.3569 - val_acc: 0.9309
Epoch 3/50
60000/60000 [==============================] - 3s 44us/step - loss: 0.5177 - acc: 0.8826 - val_loss: 0.3213 - val_acc: 0.9359
Epoch 4/50
60000/60000 [==============================] - 3s 48us/step - loss: 0.4878 - acc: 0.8883 - val_loss: 0.2923 - val_acc: 0.9442
Epoch 5/50
60000/60000 [==============================] - 3s 49us/step - loss: 0.4588 - acc: 0.8939 - val_loss: 0.2912 - val_acc: 0.9389
Epoch 6/50
60000/60000 [==============================] - 3s 48us/step - loss: 0.4513 - acc: 0.8973 - val_loss: 0.2803 - val_acc: 0.9441
Epoch 7/50
60000/60000 [==============================] - 3s 51us/step - loss: 0.4394 - acc: 0.9006 - val_loss: 0.2797 - val_acc

Rich loss: 0.1536 - acc: 0.9722 - val_loss: 0.1570 - val_acc: 0.971 on epoch 25...
The difference between trainning accuracy and test accuracy is reduced. validation 
Overfitting will see the train accuracy getting higher but the test one decreasing.